1. Dockerizing the FastAPI Backend

	1.	Create a Dockerfile for the FastAPI backend:

In [ ]:
# backend/Dockerfile

# Use an official Python runtime as a parent image
FROM python:3.9-slim

# Set the working directory in the container
WORKDIR /app

# Copy the requirements file to the working directory
COPY requirements.txt .

# Install the required dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy the FastAPI app code to the container
COPY . .

# Expose the port FastAPI runs on
EXPOSE 8000

# Command to run the FastAPI app
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8000"]

	2.	Create a requirements.txt for the backend (if you don’t have one already):

In [ ]:
fastapi
uvicorn
sqlalchemy
pydantic
python-dotenv

	3.	Build the Docker image:
Run the following command from the backend/ directory:

In [ ]:
docker build -t fastapi-backend .

2. Dockerizing the React Frontend

	1.	Create a Dockerfile for the React frontend:

In [ ]:
# frontend/Dockerfile

# Use an official Node.js runtime as a parent image
FROM node:16-alpine

# Set the working directory in the container
WORKDIR /app

# Copy the package.json and yarn.lock (or package-lock.json if using npm)
COPY package*.json ./

# Install the dependencies
RUN npm install

# Copy the rest of the frontend code
COPY . .

# Build the React app for production
RUN npm run build

# Expose the port the frontend runs on (e.g., port 3000)
EXPOSE 3000

# Start the app
CMD ["npm", "start"]

	2.	Build the Docker image:
Run the following command from the frontend/ directory:

In [ ]:
docker build -t react-frontend .

3. Docker Compose for Both Services

Next, you’ll create a docker-compose.yml file to run both the FastAPI backend and the React frontend in separate containers, but allow them to communicate with each other.

	1.	Create a docker-compose.yml file at the root of your project (outside both the frontend/ and backend/ folders):

version: '3.8'

services:
  backend:
    build:
      context: ./backend
      dockerfile: Dockerfile
    container_name: fastapi-backend
    ports:
      - "8000:8000"
    environment:
      - DATABASE_URL=sqlite:///./test.db  # Example of environment variable
    networks:
      - mynetwork

  frontend:
    build:
      context: ./frontend
      dockerfile: Dockerfile
    container_name: react-frontend
    ports:
      - "3000:3000"
    networks:
      - mynetwork

networks:
  mynetwork:
    driver: bridge

4. Configuring Communication Between FastAPI and React

	•	By default, containers in the same Docker network can communicate with each other by referring to the service names defined in the docker-compose.yml. In this case, the React app can refer to the FastAPI backend as http://backend:8000 because backend is the service name.
	•	In the React frontend, instead of http://localhost:8000, you should call http://backend:8000 to communicate with the FastAPI service.
Update your React service calls like so:

In [ ]:
// services/api.ts
import axios from 'axios';

// Note: 'backend' refers to the FastAPI service in Docker
const API_BASE_URL = 'http://backend:8000';

export const fetchItems = async () => {
  const response = await axios.get(`${API_BASE_URL}/items`);
  return response.data;
};

5. Running the Containers

	1.	Build and start both containers:
From the root of your project, where the docker-compose.yml is located, run:

In [ ]:
docker-compose up --build


	2.	Access the services:
	•	The FastAPI backend will be available at http://localhost:8000.
	•	The React frontend will be available at http://localhost:3000.
	3.	Verify that they are communicating:
When the React frontend makes requests to the backend (http://backend:8000 inside the Docker network), you should see data being fetched and displayed in your React components.

6. CORS Handling in FastAPI

To ensure your React app can interact with the FastAPI backend, you’ll need to set up CORS properly in FastAPI:

In [ ]:
# backend/app/main.py
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

# Allow the frontend to make requests to the backend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow all origins (or specify your frontend's address, e.g., "http://localhost:3000")
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
def read_root():
    return {"message": "Welcome to FastAPI"}